In [1]:
import numpy as np
import pandas as pd
import os
import statsmodels.api as sm

In [2]:
dirname=os.getcwd()
file_list=os.listdir(dirname+'\\input_data')

In [3]:
for file in file_list:
    if '权重' in file:
        file_list.remove(file)

In [4]:
all=[]
for i in file_list:
    temp=pd.read_excel('input_data\\'+i,sheet_name='规格品汇总')
    temp=temp.iloc[3:,0:3]
    temp = temp.rename(columns={'Unnamed: 2':i[:-5], 'Unnamed: 1':'规格'})
    temp.set_index(['表2. 各规格品汇总表','规格'],inplace=True)
    all.append(temp)
new_file=pd.concat(objs=all,axis=1)
#一定要转换成float32才可以进行Log操作
new_file[0:]=new_file[0:].astype('float32')
new_file=new_file[new_file!=0]
new_file.dropna(inplace=True)
new_file=np.log(new_file)
new_file.reset_index(inplace=True)

In [5]:
new_file['表2. 各规格品汇总表']=new_file.iloc[:,0].str[:7]
new_file.to_excel('new_file.xlsx')

In [6]:
grouped=new_file.groupby(['表2. 各规格品汇总表'])
def regulation(df_in):
    #reconstruct the data
    df_in=pd.melt(frame=df_in,id_vars=['规格'],value_vars=df_in.columns[2:],var_name=['location'],value_name='price')
    
    #select the y
    y=df_in['price']

    #set
    df_in=pd.get_dummies(df_in[['规格','location']])

    #select x
    x=df_in
    x=sm.add_constant(x)

    #fit the model
    model=sm.OLS(y,x)
    result=model.fit()

    #get the beta
    beta=result.params
    beta=beta.to_frame().T

    #change the type
    beta=beta.astype('float32')
    beta=np.exp(beta)
    return beta
last=grouped.apply(regulation).loc[:,'location_上饶':'location_鹰潭']
nanchang=last.loc[:,'location_南昌']
def cal_index(x):
    return x/nanchang
last=last.apply(cal_index)

D:\software\Anaconda\envs\data\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [7]:
last.to_excel('final_vision.xlsx')